## Setting up PySpark in Colab

In [2]:
# install findspark
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-2.4.4/spark-2.4.4-bin-hadoop2.7.tgz
!tar xf spark-2.4.4-bin-hadoop2.7.tgz
!pip install -q findspark

In [3]:
# setting the environment path
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.4-bin-hadoop2.7"

# import findspark
import findspark
findspark.init('spark-2.4.4-bin-hadoop2.7')

In [5]:
# create a SparkSession
from pyspark.sql import SparkSession
spark = SparkSession.builder \
    .master('local[*]') \
    .appName('Iniciando com Spark') \
    .config('spark.ui.port', '4050') \
    .getOrCreate()

In [7]:
spark

In [6]:
# downloading the file
!wget --verbose --show-progress --no-check-certificate https://raw.githubusercontent.com/jonates/opendata/master/receita_federal/receita_federal_arrecadacao_por_UF_2020.csv

--2022-09-02 16:32:06--  https://raw.githubusercontent.com/jonates/opendata/master/receita_federal/receita_federal_arrecadacao_por_UF_2020.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6216 (6.1K) [text/plain]
Saving to: ‘receita_federal_arrecadacao_por_UF_2020.csv’

receita_federal_arr 100%[===================>]   6.07K  --.-KB/s    in 0s      

2022-09-02 16:32:06 (59.5 MB/s) - ‘receita_federal_arrecadacao_por_UF_2020.csv’ saved [6216/6216]



## Starting with PySpark


In [10]:
# loading data in pyspark
df = spark.read.csv(
    path = "/content/receita_federal_arrecadacao_por_UF_2020.csv", 
    inferSchema = True, 
    header = True,
    sep = ';', 
    encoding = "UTF-8")

In [43]:
df.describe()

DataFrame[summary: string, uf: string, regiao: string, ano: string, imposto_sobre_importacao: string, imposto_sobre_exportacao: string, ipi_total: string, imposto_sobre_a_renda_total: string, irpf: string, irpj: string, imposto_s_renda_retido_na_fonte: string, imposto_s_operacoes_financeiras: string, imposto_territorial_rural: string, cofins: string, contribuicao_para_o_pis_pasep: string, csll: string, cide_combustiveis: string, cpsss_contrib_p_o_plano_de_segurid_social_serv_publico: string, outras_receitas_administradas: string]

In [13]:
df.count()

28

## Basic Commands for PySpark SQL

In [25]:
# reading data in spark.sql
df.createOrReplaceTempView('dfSql')
sql_df = spark.sql('SELECT * FROM dfSql')

In [27]:
sql_df.show(5)

+---+--------+----+------------------------+------------------------+-------------+---------------------------+------------+-------------+-------------------------------+-------------------------------+-------------------------+-------------+-----------------------------+-------------+-----------------+------------------------------------------------------+-----------------------------+
| uf|  regiao| ano|imposto_sobre_importacao|imposto_sobre_exportacao|    ipi_total|imposto_sobre_a_renda_total|        irpf|         irpj|imposto_s_renda_retido_na_fonte|imposto_s_operacoes_financeiras|imposto_territorial_rural|       cofins|contribuicao_para_o_pis_pasep|         csll|cide_combustiveis|cpsss_contrib_p_o_plano_de_segurid_social_serv_publico|outras_receitas_administradas|
+---+--------+----+------------------------+------------------------+-------------+---------------------------+------------+-------------+-------------------------------+-------------------------------+------------------

In [31]:
spark.sql("SELECT * FROM dfSql WHERE regiao = 'Norte'").show()

+---+------+----+------------------------+------------------------+------------+---------------------------+------------+-------------+-------------------------------+-------------------------------+-------------------------+-------------+-----------------------------+-------------+-----------------+------------------------------------------------------+-----------------------------+
| uf|regiao| ano|imposto_sobre_importacao|imposto_sobre_exportacao|   ipi_total|imposto_sobre_a_renda_total|        irpf|         irpj|imposto_s_renda_retido_na_fonte|imposto_s_operacoes_financeiras|imposto_territorial_rural|       cofins|contribuicao_para_o_pis_pasep|         csll|cide_combustiveis|cpsss_contrib_p_o_plano_de_segurid_social_serv_publico|outras_receitas_administradas|
+---+------+----+------------------------+------------------------+------------+---------------------------+------------+-------------+-------------------------------+-------------------------------+-------------------------+-

In [42]:
spark.sql("SELECT uf, regiao, imposto_sobre_importacao FROM dfSql \
WHERE imposto_sobre_importacao = (SELECT (MAX(imposto_sobre_importacao)) FROM dfSql)").show()

+---+------+------------------------+
| uf|regiao|imposto_sobre_importacao|
+---+------+------------------------+
| PA| Norte|             98912193,42|
+---+------+------------------------+

